In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
from selenium.webdriver.common.by import By
import pandas as pd

/Users/yamamos/Documents/research/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
item_dict = {"E461004":"ヒートテックVネックT（9分袖）",
            #  "E441620":"ヒートテッククルーネックT（9分袖）",
            #  "E462734":"ヒートテックコットンタートルネックT（極暖・9分袖）",
            #  "E450762":"ヒートテックコットンVネックT（極暖・9分袖）",
            #  "E450529":"ヒートテックコットンクルーネックT（極暖・9分袖）",
            #  "E451538":"ヒートテックコットンクルーネックT（極暖・9分袖）",
            #  "E461002":"ヒートテックVネックT（半袖）",
            #  "E441664":"ヒートテッククルーネックT（半袖）"
            }

In [5]:
# item_list = ["E461004"]
item_list = list(item_dict.keys())
print(item_list)

['E461004']


In [13]:
item_list = [item_list[-1]]

In [6]:
options = ChromeOptions()
options.add_argument("--headless")


# driver = webdriver.Chrome(ChromeDriverManager().install())

driver = webdriver.Chrome()


for item in item_list:
    driver.get(f"https://www.uniqlo.com/jp/ja/products/{item}-000/00/reviews")

    time.sleep(1)

    n = 0
    soup_len = 0

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 最下部までスクロール
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # ページが読み込まれるのを待つ

        # 新しいページの高さを取得して、スクロールが必要か判断
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # 少し上にスクロールしてから再度下にスクロール
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 500);")
            time.sleep(1)  # 少し待つ
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 再読込のために待つ

            # 再度高さを確認
            very_new_height = driver.execute_script("return document.body.scrollHeight")
            if very_new_height == new_height:
                break  # 変化がなければ終了
            else:
                last_height = very_new_height  # 更新された高さで続行
        else:
            last_height = new_height


    
    reviews = soup.find_all('p', class_='fr-ec-review__content')
    stars = soup.find_all('div', class_='fr-ec-rating-static fr-ec-rating')
    product = soup.find_all('h3', class_='fr-ec-display')
    product = product[0].get_text()


    others = soup.find_all('span', class_="fr-ec-caption fr-ec-caption--color-primary-dark fr-ec-text-align-left fr-ec-review__user-info fr-ec-review__grey-text fr-ec-caption--standard fr-ec-text-transform-normal")
    g_list = ['男性', '女性', '回答しない', 'その他']
    a_list = ['10 - 14歳','15 - 19歳','20代','30代','40代','50代','60代以上']


    gender_list = []
    age_list = []

    for t in others:
        text = t.get_text()
        
        if text in g_list:
            gender_list.append(text)
            
        elif text in a_list:
            age = t.get_text()
            age_list.append(age)

        if len(gender_list) - len(age_list) == 2:
            age_list.append('回答しない')

    print(gender_list)
    print(age_list)

    review_list = [review.get_text() for review in reviews]
    print(review_list)

    star_list = []

    for s in stars:
        single_rev_stars = s.find_all('svg')
        single_rev_stars_count = sum(1 for star in single_rev_stars if "fr-ec-star--full" in str(star))
        star_list.append(single_rev_stars_count)

    print(star_list)


    # dataframeを作成
    df = pd.DataFrame({'product': product, 'review': review_list, 'star': star_list, 'gender':gender_list, 'age':age_list})
    df.to_csv(f'../data/uniqlo_{product}.csv', index=False)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.119)
Stacktrace:
0   chromedriver                        0x00000001046b6924 chromedriver + 4368676
1   chromedriver                        0x00000001046aedb8 chromedriver + 4337080
2   chromedriver                        0x00000001042d2c04 chromedriver + 289796
3   chromedriver                        0x00000001042ad918 chromedriver + 137496
4   chromedriver                        0x0000000104339ab8 chromedriver + 711352
5   chromedriver                        0x000000010434cd08 chromedriver + 789768
6   chromedriver                        0x0000000104309ab4 chromedriver + 514740
7   chromedriver                        0x000000010430a50c chromedriver + 517388
8   chromedriver                        0x000000010467ae40 chromedriver + 4124224
9   chromedriver                        0x000000010467fc30 chromedriver + 4144176
10  chromedriver  

In [21]:
options = ChromeOptions()
options.add_argument("--headless")


# driver = webdriver.Chrome(ChromeDriverManager().install())

driver = webdriver.Chrome()
driver.get(f"https://www.uniqlo.com/jp/ja/products/E461004-000/00/reviews")

time.sleep(2)
driver.execute_script("return document.body.scrollHeight")
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
product = soup.find_all('span', class_='fr-ec-caption fr-ec-caption--color-primary-dark fr-ec-text-align-left fr-ec-review__grey-text fr-ec-review__date fr-ec-caption--standard fr-ec-text-transform-normal')

In [25]:
date = soup.find_all('span', class_="fr-ec-caption fr-ec-caption--color-primary-dark fr-ec-text-align-left fr-ec-review__grey-text fr-ec-review__date fr-ec-caption--standard fr-ec-text-transform-normal")


In [28]:
for t in p:
    print(t.get_text())

2024/5/8
2024/5/8
2024/4/18
2024/4/4
2024/3/30


In [22]:
soup.find_all('h3', class_='fr-ec-display')

[<h3 class="fr-ec-display fr-ec-display--color-primary-dark fr-ec-display--display3 fr-ec-text-align-left fr-ec-text-transform-normal fr-ec-mb-spacing-04" style="-webkit-line-clamp: 3;">ヒートテックVネックT（9分袖）</h3>]

In [69]:
len(pres_rev)

15

In [55]:
len(reviews)

552

In [50]:
reviews = soup.find_all('p', class_='fr-ec-review__content')
stars = soup.find_all('div', class_='fr-ec-rating-static fr-ec-rating')
product = soup.find_all('h3', class_='fr-ec-display')
product = product[0].get_text()

In [ ]:
<span class="fr-ec-body fr-ec-body--color-primary-dark fr-ec-body--standard fr-ec-text-align-left fr-ec-text-transform-normal" data-testid="CoreBody">552 件</span>

In [62]:
_ = soup.select('#root > section.fr-ec-layout-wrapper.fr-ec-template-ilp__main-content-min-height > div:nth-child(3) > section > div > div.fr-ec-layout.fr-ec-layout--span-4-sm.fr-ec-layout--span-12-md.fr-ec-layout--span-12-lg.fr-ec-filter-layout-utility-bar > div > div.fr-ec-sticky-container > div > section > span')

In [64]:
rev_len = str(_).split(" ")[-2].split(">")[-1]

['[<span',
 'class="fr-ec-body',
 'fr-ec-body--color-primary-dark',
 'fr-ec-body--standard',
 'fr-ec-text-align-left',
 'fr-ec-text-transform-normal"',
 'data-testid="CoreBody">552',
 '件</span>]']

In [59]:
soup.text

"ユニクロ公式 | ヒートテックタートルネックT（9分袖）のレビュー\nユニクロトップへUNIQLOMENインナー・下着ヒートテック9分袖ヒートテックタートルネックT（9分袖）お客様のレビューお客様のレビュー一覧ヒートテックタートルネックT（9分袖）4.5(552)レビューを書くお客様の評価36013438128お客様の着用感小さいちょうどよい大きい552 件並べ替え投稿日時が新しい順役に立った順評価が高い順評価が低い順絞り込み毎年買ってます2024/4/5購入サイズ: Lお客様の着用感: ちょうどよい毎年\u3000冬前に買ってます\u3000主人の仕事用ですがちょうどいいみたいですうさぎさん男性50代身長: 171 - 175cm岡山県不適切を通報役に立った0よかった2024/4/2購入サイズ: Mお客様の着用感: ちょうどよい中学生の息子のスキーように購入。私も後から使ってますが、着心地が良いですわいけい女性大阪府不適切を通報役に立った0冬のマストアイテム2024/3/13購入サイズ: Mお客様の着用感: ちょうどよい冬の必需品です。\nこれを着ると寒さが和らぎ\nないと冬を越せません。ナナシー回答しない東京都不適切を通報役に立った0改良されていますね。2024/3/12購入サイズ: Mお客様の着用感: ちょうどよい黒のＭサイズ。161cm、61㎏の私にジャストフィットです。\nヒートテックのタートルネックはかなり前に1枚（中国製、アクリル40%）、数年前にもう1枚（バングラデシュ製、ポリエステル35％）購入していますが、今回のものが着ていて一番暖かく感じます。\n以前のものは着るとすぐに首回りがだらんと伸びましたが、今回のものは大丈夫です。pogopapa男性60代以上身長: 161 - 165cm体重: 61 - 65kg足のサイズ: 25.0cm埼玉県不適切を通報役に立った0再販売希望2024/3/5購入サイズ: XLお客様の着用感: ちょうどよいXLサイズの再販を希望します。\n極暖だと厚過ぎてインナーに向きません。\nクルーネックだと首回りが寒いです。\nちょうどいい薄さのちょうどいいタートルネックインナーはこれしか無いです。\n何卒よろしくお願いします。強く再販希望男性30代身長: 171 - 175cm体重: 81 - 85kg足のサイズ: 26

In [51]:
others = soup.find_all('span', class_="fr-ec-caption fr-ec-caption--color-primary-dark fr-ec-text-align-left fr-ec-review__user-info fr-ec-review__grey-text fr-ec-caption--standard fr-ec-text-transform-normal")
g_list = ['男性', '女性', '回答しない', 'その他']
a_list = ['10 - 14歳','15 - 19歳','20代','30代','40代','50代','60代以上']


gender_list = []
age_list = []

for t in others:
    text = t.get_text()
    
    if text in g_list:
        gender_list.append(text)
        
    elif text in a_list:
        age = t.get_text()
        age_list.append(age)

    if len(gender_list) - len(age_list) == 2:
        age_list.append('回答しない')

print(gender_list)
print(age_list)

review_list = [review.get_text() for review in reviews]
print(review_list)

star_list = []

for s in stars:
    single_rev_stars = s.find_all('svg')
    single_rev_stars_count = sum(1 for star in single_rev_stars if "fr-ec-star--full" in str(star))
    star_list.append(single_rev_stars_count)

print(star_list)

['男性', '女性', '回答しない', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '女性', '男性', '男性', '男性', '男性', '男性', '男性', '女性', '男性', '男性', '男性', '男性', '男性', '男性', 'その他', '男性', '男性', '女性', '男性', '女性', '男性', '男性', '男性', '男性', '男性', '男性', '女性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '女性', '男性', '男性', '男性', '女性', '男性', '男性', '男性', '男性', '男性', '男性', '女性', '男性', '男性', '女性', '男性', '女性', '男性', '男性', '男性', '回答しない', '男性', '女性', '女性', '男性', '男性', '女性', '女性', '女性', '男性', '男性', '男性', '女性', '男性', '男性', '男性', '男性', '女性', '男性', '男性', '女性', '女性', '男性', '男性', '男性', '女性', '男性', '男性', '回答しない', '男性', '男性', '男性', '男性', '男性', '回答しない', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '女性', '男性', '女性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '回答しない', '男性', '男性', '男性', '女性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', '男性', 'その他', '男性', '男性', '男性', '回答しない', '

In [52]:
# dataframeを作成
df = pd.DataFrame({'product': product, 'review': review_list, 'star': star_list, 'gender':gender_list, 'age':age_list})
df

,product,review,star,gender,age
0,ヒートテックタートルネックT（9分袖）,毎年 冬前に買ってます 主人の仕事用ですがちょうどいいみたいです,3,男性,50代
1,ヒートテックタートルネックT（9分袖）,中学生の息子のスキーように購入。私も後から使ってますが、着心地が良いです,4,女性,回答しない
2,ヒートテックタートルネックT（9分袖）,冬の必需品です。\nこれを着ると寒さが和らぎ\nないと冬を越せません。,4,回答しない,回答しない
3,ヒートテックタートルネックT（9分袖）,黒のＭサイズ。161cm、61㎏の私にジャストフィットです。\nヒートテックのタートルネック...,5,男性,60代以上
4,ヒートテックタートルネックT（9分袖）,XLサイズの再販を希望します。\n極暖だと厚過ぎてインナーに向きません。\nクルーネックだと...,5,男性,30代
...,...,...,...,...,...
547,ヒートテックタートルネックT（9分袖）,仕事のインナーとして着ています微妙な温度差の時に着れるので重宝してます,4,男性,20代
548,ヒートテックタートルネックT（9分袖）,これ一枚でも、ジャケット羽織っても9分丈なのでスッキリ,5,男性,回答しない
549,ヒートテックタートルネックT（9分袖）,毎シーズン購入している定番品。薄くて暖かいので重ね着することで一冬を安心して過ごせます。１シ...,5,男性,60代以上
550,ヒートテックタートルネックT（9分袖）,去年、買いそびれていたのでこれからフル活用していきます！,5,男性,50代


In [53]:
df.to_csv(f'../data/uniqlo_{product}.csv', index=False)

In [36]:
review_list = [review.get_text() for review in reviews]
print(review_list)

['毎年\u3000冬前に買ってます\u3000主人の仕事用ですがちょうどいいみたいです', '中学生の息子のスキーように購入。私も後から使ってますが、着心地が良いです', '冬の必需品です。\nこれを着ると寒さが和らぎ\nないと冬を越せません。', '黒のＭサイズ。161cm、61㎏の私にジャストフィットです。\nヒートテックのタートルネックはかなり前に1枚（中国製、アクリル40%）、数年前にもう1枚（バングラデシュ製、ポリエステル35％）購入していますが、今回のものが着ていて一番暖かく感じます。\n以前のものは着るとすぐに首回りがだらんと伸びましたが、今回のものは大丈夫です。', 'XLサイズの再販を希望します。\n極暖だと厚過ぎてインナーに向きません。\nクルーネックだと首回りが寒いです。\nちょうどいい薄さのちょうどいいタートルネックインナーはこれしか無いです。\n何卒よろしくお願いします。', '冬の次期、下着も兼ねて、気持ちよく着ています。\n４、5年前のように、色を増やし、大々的に販売して欲しい。', '薄地で温かいので下着として愛用しています。\n白でも黒でも良し。冬の定番です。', '冬の日常着としてとても良いと思います。店舗に無くて、オンラインストアで購入しています。黒のみだったので、他の色も欲しいです。', '身長177cm体重65kgでLサイズでジャストです。\nタートルネックは店舗では見つけられなかったので、オンラインストアで購入しました。', 'BLACKのМサイズを購入しました。\n毎年購入していて、毎年形や生地がアップデートされてるのか良さを感じていたが、今年のはどーなの？！生地が薄い！爪があたったらスグに破れてしまいそう。ビックリした。値上げしてコノ商品は残念な気分です。', 'インナーとしてとても優秀、睡眠時の首冷えを抑えることもできる。文句なしの名作', 'クルーネックの下に着るのに最適です。\nもう少し袖丈が長ければ更に良かった。', 'ブラック購入、使用を続けていてネック部分が緩んできたので新たに購入。サイズもMからLにしてみました。まだ着用していないので分からないですが、以前の同じものはもう1年以上愛用しています。', 'とにかく、暖かいです。肌触りも違和感がありません。色違いで、また購入したいです。', '外仕事の主人の

In [37]:
star_list = []

for s in stars:
    single_rev_stars = s.find_all('svg')
    single_rev_stars_count = sum(1 for star in single_rev_stars if "fr-ec-star--full" in str(star))
    star_list.append(single_rev_stars_count)

print(star_list)

[3, 4, 4, 5, 5, 5, 5, 5, 5, 3, 5, 4, 5, 5, 4, 5, 3, 5, 5, 5, 5, 4, 5, 4, 3, 5, 5, 3, 5, 3, 2, 4, 5, 5, 1, 5, 5, 4, 5, 4, 4, 5, 5, 5, 4, 4, 5, 5, 5, 5]
